<a href="https://colab.research.google.com/github/AliKarimiENT/NLP_HW1/blob/main/NLPHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reading txt file and getting verses

Here we need to do some processes

In [27]:
# with this function we check that it contains number in the text or not
def hasNumbers(inputString) :
  for character in inputString :
    if character.isdigit() :
      return True
  return False

def hasDash(inputString) :
  for character in inputString :
    if character == '-' :
      return True
  return False 


with open('/content/ShamsDaftar6.doc_filename=ShamsDaftar6.txt') as f:
  dataSet = f.readlines()

verses = []
for line in dataSet :
  if line != ' ' and line != '\n' and hasNumbers(line) == False  and hasDash(line)==False:
    if dataSet.index(line) != 0 :
      verses.append(line.replace('\t',' '))


for i in range(20):
  print(verses[i])


گرم سیم و درم بودی مرا مونس چه کم بودی وگر یارم فقیرستی ز زر فارغ چه غم بودی

خدایا حرمت مردان ز دنیا فارغش گردان از آن گر فارغستی او ز پیش من چه کم بودی

نگارا گر مرا خواهی وگر همدرد و همراهی مکن آه و مخور حسرت که بختم محتشم بودی

بتا زیبا و نیکویی رها کن این گدارویی اگر چشم تو سیرستی فلک ما را حشم بودی

ز طمع آدمی باشد که خویش از وی چو بیگانه است وگر او بی طمع بودی همه کس خال و عم بودی

بیا چون ما شو ای مه رو نه نعمت جو نه دولت جو گر ابلیس این چنین بودی شه و صاحب علم بودی

از ابلیسی جدا بودی سقط او را ثنا بودی جفا او را وفا بودی سقم او را کرم بودی

زهی اقبال درویشی زهی اسرار بی خویشی اگر دانستیی پیشت همه هستی عدم بودی

جهانی هیچ و ما هیچان خیال و خواب ما پیچان وگر خفته بدانستی که در خوابم چه غم بودی

خیالی بیند این خفته در اندیشه فرورفته وگر زین خواب آشفته بجستی در نعم بودی

یکی زندان غم دیده یکی باغ ارم دیده وگر بیدار گشتی او نه زندان نی ارم بودی

امیر دل همی گوید تو را گر تو دلی داری که عاشق باش تا گیری ز نان و جامه بیزاری

تو را گر قحط نان باشد کند عشق تو خبازی وگر گم گشت دستارت ک

Get list of stop words

In [35]:
import re
with open('/content/stopwords.txt') as f:
  stops = f.readlines()
stopWords = []
for stop in stops :
  stopWords.append(re.sub('\n','',stop.replace('\ufeff','')))
print(stopWords)

['!', '"', '#', '(', ')', '*', ',', '-', '.', '/', ':', '[', ']', '«', '»', '،', '؛', '؟اتفاقا', 'احتراما', 'احتمالا', 'اري', 'آري', 'از', 'ازجمله', 'اساسا', 'است', 'اش', 'اشكارا', 'آشكارا', 'اصلا', 'اصولا', 'اغلب', 'اكثرا', 'اكنون', 'الان', 'الآن', 'البته', 'ام', 'اما', 'امد', 'آمد', 'امدم', 'آمد', 'امدن', 'آمدن', 'امدند', 'آمدند', 'امده', 'آمده', 'امدي', 'آمدی', 'امديد', 'آمدید', 'امديم', 'آمدیم', 'امروزه', 'امسال', 'امشب', 'ان', 'آن', 'اند', 'انشاالله', 'انصافا', 'انطور', 'آنطور', 'انقدر', 'آنقدر', 'انها', 'آنها', 'انچنان', 'آنچنان', 'انگار', 'او', 'اورد', 'آورد', 'اوردم', 'آوردم', 'اوردن', 'آوردن', 'اوردند', 'آوردند', 'اورده', 'آورده', 'اوردي', 'آوردی', 'اورديد', 'آوردید', 'اورديم', 'آوردیم', 'اورم', 'آورم', 'اورند', 'آورند', 'اوري', 'آوری', 'اوريد', 'آورید', 'اوريم', 'آوری', 'اولا', 'اي', 'آی', 'ايا', 'آیا', 'ايد', 'ايشان', 'ايم', 'اين', 'ايند', 'آيند', 'اينطور', 'اينقدر', 'اينك', 'اينها', 'اينچنين', 'اينگونه', 'ايي', 'آیی', 'اييد', 'آیید', 'اييم', 'آییم', 'اگر', 'با', 'بار', 'بار

Remove stop words from verses

In [36]:
#list of whole words of concept
words = []

for verse in verses :
  for word in verse.split() :
   if word not in stopWords :
     words.append(word)

print(words)

['گرم', 'سیم', 'درم', 'بودی', 'مرا', 'مونس', 'کم', 'بودی', 'وگر', 'یارم', 'فقیرستی', 'ز', 'زر', 'فارغ', 'غم', 'بودی', 'خدایا', 'حرمت', 'مردان', 'ز', 'دنیا', 'فارغش', 'گردان', 'گر', 'فارغستی', 'ز', 'پیش', 'کم', 'بودی', 'نگارا', 'گر', 'مرا', 'خواهی', 'وگر', 'همدرد', 'همراهی', 'مکن', 'آه', 'مخور', 'حسرت', 'که', 'بختم', 'محتشم', 'بودی', 'بتا', 'زیبا', 'نیکویی', 'رها', 'کن', 'این', 'گدارویی', 'چشم', 'سیرستی', 'فلک', 'حشم', 'بودی', 'ز', 'طمع', 'آدمی', 'که', 'خویش', 'وی', 'بیگانه', 'وگر', 'بی', 'طمع', 'بودی', 'کس', 'خال', 'عم', 'بودی', 'بیا', 'شو', 'ای', 'مه', 'نعمت', 'جو', 'دولت', 'جو', 'گر', 'ابلیس', 'این', 'چنین', 'بودی', 'شه', 'صاحب', 'علم', 'بودی', 'ابلیسی', 'جدا', 'بودی', 'سقط', 'ثنا', 'بودی', 'جفا', 'وفا', 'بودی', 'سقم', 'کرم', 'بودی', 'زهی', 'اقبال', 'درویشی', 'زهی', 'اسرار', 'بی', 'خویشی', 'دانستیی', 'پیشت', 'هستی', 'عدم', 'بودی', 'جهانی', 'هیچ', 'هیچان', 'خیال', 'خواب', 'پیچان', 'وگر', 'خفته', 'بدانستی', 'که', 'خوابم', 'غم', 'بودی', 'خیالی', 'بیند', 'این', 'خفته', 'اندیشه', 'فرورفته